This code does the wavelet transform for the whole event

In [77]:
from __future__ import division
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pycwt as wavelet
from datetime import datetime, timedelta
import matplotlib.colors as colors
from matplotlib.collections import LineCollection

In [78]:
# calculate the horizontal component of the magnetic field
def horizontal(df):
    Bn = df['dbn_nez'].to_numpy()
    Be = df['dbe_nez'].to_numpy()
    Bh = np.sqrt(Bn**2+Be**2)
    return Bh

In [90]:
# function that calculates wavelet transform
def wavepower(dat):
    
    dt = 60.0 #timestep

    mother = wavelet.Morlet(6)
    s0 = 2 * dt  # Starting scale, in this case 2 * 60 s = 2 mins
    dj = 1 / 12  # Twelve sub-octaves per octaves
    J = 9 / dj  # Nine powers of two with dj sub-octaves

    # wavelet transform
    wave, scales, freqs, coi, fft, fftfreqs = wavelet.cwt(dat, dt, dj, s0, J,
                                                        mother)
    # inverse wavelet
    iwave = wavelet.icwt(wave, scales, dt, dj, mother) 

    power = (np.abs(wave)) ** 2

    power /= scales[:, None]

    return power, freqs

In [80]:
# function that calculates the power in the pc5 range
def pc5power(freqs, power):
    a = np.where((2e-3 <= freqs) & (freqs <= 7e-3))
    low = a[0][0]
    up = a[0][-1]
    
    powers = np.sum(power[low:up+1,:],axis=0)

    return powers

In [91]:
# read data
df = pd.read_csv('filename.csv')

#change times to datetime
df['Date_UTC'] = pd.to_datetime(df['Date_UTC'], format='%Y-%m-%dT%H:%M:%S')

# get the names of all the stations
names = df.groupby('IAGA').count().index.to_numpy()

dfs = []
lats = []
lons = []

# go through all the stations 
for i in names:

   # get the data from one station
   a = df.groupby('IAGA').get_group(i)

   Bh = horizontal(a)
   time = a['Date_UTC']
   ser = pd.Series(data=Bh, index=time)
   
   # maximum number of consecutive nans
   nans = ser.isnull().astype(int).groupby(ser.notnull().astype(int).cumsum()).sum().max()

   # if data gap is larger than 40 mins or data starts with a zero, don't include it
   if nans > 40 or ser.isna()[0] == True:
      continue

   # interpolate possible nans
   ser = ser.interpolate()
   ser.resample('1s').fillna('bfill')
   B = ser.to_numpy()
   
   # do wavelet for the station
   p, f = wavepower(B)
   # take the pc5 frequencies
   pc5 = pc5power(f, p) 
   # save the pc5 powers into a dataframe
   s = {'pc5': pc5, 'MLT': a['MLT'].to_numpy(), 'MLAT': a['MAGLAT'].to_numpy()}
   dfs.append(pd.DataFrame(data=s, index=time))

   # save coordinates
   lats.append(a['GEOLAT'].iloc[0])
   lons.append(a['GEOLON'].iloc[0])

In [92]:
# add all the stations together
con = pd.concat(dfs)

# global mean of the pc5 ULF power
global_ULF = con.groupby('Date_UTC')['pc5'].mean()
time = global_ULF.index

glbl_ULF = pd.Series(global_ULF.values, name='ULF power', index=time)

# take 1 h mean
hmean = glbl_ULF.resample('1H').mean()

In [93]:
# sort the ULF powers into four MLT sectors

idx = pd.date_range(min(time), max(time), freq='T')

# ULF power in the night sector
MLT213 = con[((con['MLT']>= 21) & (con['MLT'] <= 24)) | ((con['MLT']>= 0) & (con['MLT'] < 3))]
night = MLT213.groupby('Date_UTC')['pc5'].mean()
# number of stations
nightstat = MLT213.groupby('Date_UTC').size() 

# if dates are missing, add zeros to them
if len(night) != len(time):
    night = night.reindex(idx, fill_value=0)
    nightstat = nightstat.reindex(idx, fill_value=0)

# ULF power in the dawn sector
MLT39 = con[(con['MLT'] >= 3) & (con['MLT'] < 9)]
dawn = MLT39.groupby('Date_UTC')['pc5'].mean()
# number of stations
dawnstat = MLT39.groupby('Date_UTC').size()

# if dates are missing, add zeros to them
if len(dawn) != len(time):
    dawn = dawn.reindex(idx, fill_value=0)
    dawnstat = dawnstat.reindex(idx, fill_value=0)

# ULF power in the day sector
MLT915 = con[(con['MLT'] >= 9) & (con['MLT'] < 15)]
day = MLT915.groupby('Date_UTC')['pc5'].mean()
# number of stations
daystat = MLT915.groupby('Date_UTC').size()

# if dates are missing, add zeros to them
if len(day) != len(time):
    day = day.reindex(idx, fill_value=0)
    daystat = daystat.reindex(idx, fill_value=0)

# ULF power in the dusk sector
MLT1521 = con[(con['MLT'] >= 15) & (con['MLT'] < 21)]
dusk = MLT1521.groupby('Date_UTC')['pc5'].mean()
# number of stations
duskstat = MLT1521.groupby('Date_UTC').size()

# if dates are missing, add zeros to them
if len(dusk) != len(time):
    dusk = dusk.reindex(idx, fill_value=0)
    duskstat = duskstat.reindex(idx, fill_value=0)

Next cells are for plotting

In [86]:
# GET ULF INDICES for plotting
import read_virbo_ULF as VULF

#start
year_s = 2015
month_s = 5
day_s = 12
start_time = str(year_s)+'-'+str(month_s)+'-'+str(day_s)

#end
year_e = 2015
month_e = 5
day_e = 18
end_time = str(year_e)+'-'+str(month_e)+'-'+str(day_e)

startTime = datetime(year_s, month_s, day_s)
stopTime = datetime(year_e, month_e, day_e)

timeTgr, Tgr, Tgeo , Tgr_detrend, Tgeo_detrend= VULF.read_ULF_asc_data(startTime, stopTime)

dates = []
for i in timeTgr:
    dates.append(datetime.fromtimestamp(i))

In [87]:
# function to plot a multicolor line
def multicolor(y, s, ax):
    xval = mdates.date2num(global_ULF.index)

    points = np.array([xval, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    cmap = colors.LinearSegmentedColormap.from_list("", ["pink","hotpink","deeppink","mediumvioletred"])#["lavender","plum","darkviolet"])
    s = np.ma.masked_where(s<1,s)
    cmap.set_bad(color='white')
    lc = LineCollection(segments, cmap=cmap)#p.get_cmap(cmap))
    lc.set_array(s)
    lc.set_linewidth(3)

    line = ax.add_collection(lc)

    ax.set_ylim([1e-5, 1e5])
    ax.axhline(1e-3, c='grey', alpha=0.4, ls='--')
    ax.axhline(1e-1, c='grey', alpha=0.4, ls='--')
    ax.axhline(1e1, c='grey', alpha=0.4, ls='--')
    ax.axhline(1e3, c='grey', alpha=0.4, ls='--')
    
    return line

In [ ]:
# plot ULF data
fontsize = 20

fig, ax = plt.subplots(7, 1, figsize=(21,25), sharex='all')
fig.set(facecolor='white')
fig.suptitle('Magnetic latitude: 60–70', fontsize=20)

l = multicolor(night, nightstat, ax[0])
ax[0].set_ylabel('Night', fontsize=fontsize)
ax[0].set_yscale('log')

multicolor(dawn, dawnstat, ax[1])
ax[1].set_ylabel('Dawn', fontsize=fontsize)
ax[1].set_yscale('log')

multicolor(day, daystat, ax[2])
ax[2].set_ylabel('Day', fontsize=fontsize)
ax[2].set_yscale('log')

multicolor(dusk, duskstat, ax[3])
ax[3].set_ylabel('Dusk', fontsize=fontsize)
ax[3].set_yscale('log')

ax[4].plot(time, global_ULF, color='darkviolet')
ax[4].set_ylabel('mean pc5 (min)', fontsize=fontsize)
ax[4].axhline(1e-3, c='grey', alpha=0.4, ls='--')
ax[4].axhline(1e-1, c='grey', alpha=0.4, ls='--')
ax[4].axhline(1e1, c='grey', alpha=0.4, ls='--')
ax[4].axhline(1e3, c='grey', alpha=0.4, ls='--')
ax[4].set_yscale('log')

ax[5].plot(hmean.index, hmean.values, color='darkviolet')
ax[5].set_ylabel('mean pc5 (h)', fontsize=fontsize)

ax[6].plot(dates, Tgr, color='darkviolet')
ax[6].set_ylabel(r'$\rm{T_{gr}}$', fontsize=fontsize)
ax[6].set_xlim([time.min(), time.max()])
ax[6].tick_params(axis='x', labelrotation = 40)
ax[6].xaxis.set_major_formatter(mdates.DateFormatter('%y-%m-%d %H:%M:%S'))
   
plt.rc('xtick', labelsize=16)    
plt.rc('ytick', labelsize=15) 

cax = plt.axes([0.92, 0.57, 0.015, 0.3])
fig.colorbar(l,cax=cax)

plt.subplots_adjust(hspace=0.0)
fig.align_ylabels()
fig.subplots_adjust(top=0.95)
plt.show()